# Manifold GP Supervised Learning via Precision Matrix

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import numpy as np
import torch
import gpytorch
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP

Select the device: GPU or CPU.

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset

In [3]:
data_path = files('manifold_gp.data').joinpath('dumbbell.msh')
data = np.loadtxt(data_path)
sampled_x = torch.from_numpy(data[:, :2]).float().to(device)
sampled_y = torch.from_numpy(data[:, -1][:, np.newaxis]).float().to(device)
(m, n) = sampled_x.shape

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold

In [4]:
manifold_noise = 0.01
sampled_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth

In [5]:
function_noise = 0.01
sampled_y = sampled_y + function_noise * torch.randn(m, 1).to(device)

## Training Dataset

In [6]:
num_train = 100
train_idx = torch.randint(m, (num_train,))
train_x = sampled_x[train_idx, :]
train_y = sampled_y[train_idx, :]

## Test Dataset

In [7]:
num_test = 50
test_idx = torch.randint(m, (num_test,))
test_x = sampled_x[test_idx, :]
test_y = sampled_y[test_idx]

## Initialize Kernel

In [8]:
nu = 2
neighbors = 5
modes = 10
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(sampled_x, nu, neighbors, modes))

## Initialize Likelihood and GP Model

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = RiemannGP(sampled_x, sampled_y, likelihood, kernel).to(device)

## Train Model

In [10]:
lr = 1e-1
iters = 100
verbose = True
model.manifold_informed_train(lr, iters, verbose)

likelihood.noise_covar.raw_noise: -5.000 covar_module.raw_outputscale: 0.000 covar_module.base_kernel.raw_lengthscale: -1.000 covar_module.base_kernel.raw_epsilon: -2.000 Iteration: 0, Loss: 35944.164, Noise Variance: 0.007, Signal Variance: 0.693, Lengthscale: 0.313, Epsilon: 0.127
Iteration: 1, Loss: 22931.672, Noise Variance: 0.008, Signal Variance: 0.744, Lengthscale: 0.341, Epsilon: 0.116
Iteration: 2, Loss: 14654.353, Noise Variance: 0.008, Signal Variance: 0.797, Lengthscale: 0.370, Epsilon: 0.105
Iteration: 3, Loss: 9474.073, Noise Variance: 0.009, Signal Variance: 0.848, Lengthscale: 0.400, Epsilon: 0.096
Iteration: 4, Loss: 6229.480, Noise Variance: 0.010, Signal Variance: 0.899, Lengthscale: 0.430, Epsilon: 0.088
Iteration: 5, Loss: 4172.619, Noise Variance: 0.010, Signal Variance: 0.948, Lengthscale: 0.459, Epsilon: 0.081
Iteration: 6, Loss: 2844.188, Noise Variance: 0.011, Signal Variance: 0.994, Lengthscale: 0.487, Epsilon: 0.074
Iteration: 7, Loss: 1967.370, Noise Varian

Iteration: 76, Loss: -191.494, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.909, Epsilon: 0.067
Iteration: 77, Loss: -191.516, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.909, Epsilon: 0.067
Iteration: 78, Loss: -191.537, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.909, Epsilon: 0.067
Iteration: 79, Loss: -191.557, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.909, Epsilon: 0.067
Iteration: 80, Loss: -191.576, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.910, Epsilon: 0.068
Iteration: 81, Loss: -191.595, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.910, Epsilon: 0.068
Iteration: 82, Loss: -191.613, Noise Variance: 0.018, Signal Variance: 1.590, Lengthscale: 0.910, Epsilon: 0.068
Iteration: 83, Loss: -191.630, Noise Variance: 0.018, Signal Variance: 1.591, Lengthscale: 0.910, Epsilon: 0.068
Iteration: 84, Loss: -191.647, Noise Variance: 0.018, Signal Variance: 1.591, Lengthscale: 0.910